In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from matplotlib.dates import AutoDateLocator
from matplotlib.dates import ConciseDateFormatter

In [4]:
%run '/Volumes/GoogleDrive/My Drive/Estudos/IC/Henrique/Programs/RMlicelPython/profile_read_py.py'

# Reading multiple data (mean signal)

The idea here is to iterate over the data files in a month directory.

In [32]:
import os

month = '/Users/costaleirbag/floresta.if.usp.br/ftp/old/lidar/data/2014/3'

# Primeiro testo um arquivo pra saber o tamanho do array gerado, e ai gerar o dataframe
# ja certo.

days_folder = os.listdir(month)
days_folder.sort()
names = []

#os.mkdir(f'Plots/Month{os.path.basename(month)}_2014')

# Filtrando apenas as pastas
        
for day in days_folder[1:]:
    directory = month + f'/{day}'
    if os.path.isdir(directory):
        files = os.listdir(directory)
        files.sort()
        n_files = len(files)
        print(f'Number of files at day {day}: {n_files}')

        array_size = len(profile_read(directory + f'/{files[0]}')[1])
        data    = np.zeros((array_size, n_files))
        date    = []

        cont    = 0
        errors  = []
        nodata  = []

        for filename in files:
            #print(f'Arquivo: {cont+1}', end='\r')
            path = os.path.join(directory, filename)
            try:
                head, phy, raw = profile_read(path)
                date_str = head['datei'] + ' ' + head['houri']
                date.append(datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S"))
                data[:, cont] = phy[:, 0]
            except ValueError as e:
                nodata.append(str(cont))
                #print(f'Value error happened at file {cont} in day {day}.')
                #print(f"{e}, {e.__class__}")
                data[:, cont] = np.nan
                date_str = head['datei'] + ' ' + head['houri']
                date.append(datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S"))
            # except OverflowError as e:
            #     errors.append(str(cont))
            #     #print(f"Overflow error happened at file {cont}.")
            #     #print(f"{e}, {e.__class__}")
            #     data[:, cont] = np.nan
            #     date_str = head['datei'] + ' ' + head['houri']
            #     date.append(datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S"))
            cont += 1
            #print(data)

        #date = np.array(date)

        # errors_str = ', '.join(errors)
        nodata_str = ', '.join(nodata)
        # print(f'\n  All files with overflow: {errors_str}.\n')
        if len(nodata) != 0:
            print(f'\n  All files with no data: {nodata_str}.\n')



        bg_vec = np.mean(data[-500:], axis=0)
        data_nobg = data - bg_vec

        alt = np.arange(1, data.shape[0] + 1, 1)*7.5

        data_pr2 = pd.DataFrame((data_nobg.T * alt * alt).T, columns=date)
        data_pr2['Altitude'] = alt
        data_pr2.set_index('Altitude', inplace=True)


        # Tirando medias de 5 em 5 min
        data_pr2.columns = pd.to_datetime(data_pr2.columns)
        data_pr2 = data_pr2.resample('5T', axis=1).mean()
        data_pr2[data_pr2 < 0] = np.nan

        fig, ax = plt.subplots(figsize = (12,9))

        psm = ax.pcolormesh(data_pr2.columns, alt*1e-3, (data_pr2), vmin = 0, vmax=1e7, cmap='jet')

        cbar = fig.colorbar(psm, ax=ax, label = "Range Corrected Signal [a. u.]")

        ax.set_xlabel("Date", fontsize=20)

        ax.set_ylabel("Altitude (km)", fontsize=20)

        ax.set_ylim(0, 15)

        cbar.ax.tick_params(labelsize=15)

        ax.figure.axes[-1].yaxis.label.set_size(20)

        locator   = AutoDateLocator(minticks=2, maxticks=8)
        formatter = ConciseDateFormatter(locator)
        ax.xaxis.set_major_locator(locator)
        ax.xaxis.set_major_formatter(formatter)

        plt.savefig(f'Plots/Month{os.path.basename(month)}_2014/Day{day}.png', dpi=100)

        plt.close()

Number of files at day 01: 3724
Number of files at day 02: 7202
Number of files at day 03: 7155
Number of files at day 04: 7216
Number of files at day 05: 7220
Number of files at day 06: 7162
Number of files at day 07: 7192
Number of files at day 08: 4347
Number of files at day 09: 7207
Number of files at day 10: 7201
Number of files at day 11: 7220
Number of files at day 12: 7215
Number of files at day 13: 7215
Number of files at day 14: 7215


In [31]:
days_folder

['01.zip',
 '01.zip.md5',
 '02.zip',
 '02.zip.md5',
 '03.zip',
 '03.zip.md5',
 '04.zip',
 '04.zip.md5',
 '05.zip',
 '05.zip.md5',
 '06.zip',
 '06.zip.md5',
 '07.zip',
 '07.zip.md5',
 '08.zip',
 '08.zip.md5',
 '09.zip',
 '09.zip.md5',
 '10.zip',
 '10.zip.md5',
 '11.zip',
 '11.zip.md5',
 '12.zip',
 '12.zip.md5',
 '13.zip',
 '13.zip.md5',
 '14.zip',
 '14.zip.md5',
 '15.zip',
 'index.html',
 'index.html?C=D;O=A',
 'index.html?C=M;O=A',
 'index.html?C=N;O=D',
 'index.html?C=S;O=A']